In [1]:
import os
import datetime
import random

import pandas as pd
import numpy as np

# Read and concat files

In [3]:
path = '../../CerData/'
output_file = 'All_Files.csv'
files = os.listdir(path)
input_files = [file for file in files if '.txt' in file]

In [3]:
df = pd.DataFrame()
for input_file in input_files:
    print(input_file)
    data = pd.read_csv(path+input_file, header = None, sep = ' ')\
    .rename(columns = {0: 'ID', 1:'Date', 2:'Consumption'})    
    df = pd.concat([df, data], ignore_index=True)

File1.txt
File2.txt
File3.txt
File4.txt
File5.txt
File6.txt


In [4]:
# WRITE
# df.to_csv(path+'All_Files.csv', index = False)

In [5]:
# READ
# df = pd.read_csv(path+output_file)

# Fix Missing Dates

In [6]:
df['hours'] = df['Date']%100
df1 =df.loc[df['hours']<=48]

In [7]:
df1['Date'].min(), df1['Date'].max()

(19501, 73048)

In [8]:
daycodes = []
for i in range(195, 730+1):
    for j in range(1, 48 +1):
        daycodes.append(i*100+j)
len(daycodes)

25728

In [9]:
dates_df = df1['Date'].unique()
len(dates_df), min(dates_df), max(dates_df)

(25726, 19501, 73048)

In [31]:
missing_dates = list(set(daycodes).difference(set(dates_df)))
missing_dates

[45202, 45203]

In [11]:
a_dic = {'ID':1000, 'Date': missing_dates}
df2 = pd.DataFrame(a_dic)
df3 = pd.concat([df1, df2], ignore_index=True, sort=False)

In [12]:
# ID =1000
# Date = 4502

In [13]:
df3 = df3[['ID', 'Date', 'Consumption']]

In [14]:
df3.to_csv(path+'ed_'+output_file)

In [4]:
df3 = pd.read_csv(path+'ed_'+output_file)

# Aggregate DataFrame

In [5]:
# tms = df.pivot(index='ID', columns = 'Date',values = 'Consumption')
d = df3.groupby(['ID', 'Date'])['Consumption'].sum()
tms = d.unstack()

In [ ]:
tms[45202].loc[1000] = float('nan')
tms[45203].loc[1000] = float('nan')

In [7]:
# tms.to_csv(path+'ed_Timeseries.csv')

In [53]:
tms = pd.read_csv(path+'ed_Timeseries', index_col='ID')

# Handle NaNs

In [61]:
def fill_na(row):
    min_ = row.min()
    med_ = row.median()
    std_ = row.std()
    mask = row.isna()
    items =len(row[mask])
    a = (min_ - med_)/std_
    b = 1
    row[mask] =[med_+random.uniform(a, b)*std_ for i in range(0,items)]    
    return row

In [19]:
#tms = tms.apply(lambda row: fill_na(row) if sum(row.isna())>0 else row)

In [62]:
tms = tms.apply(fill_na, axis = 1)

In [63]:
sum(tms.isna().any(axis = 1))

0

In [67]:
tms.to_csv(path+'clean_Timeseries.csv')

In [72]:
stats = tms.apply(pd.DataFrame.describe, axis = 1)
stats

,count,mean,std,min,25%,50%,75%,max
ID,,,,,,,,
1000,25728.0,0.214405,0.367892,0.018,0.027000,0.09100,0.221250,4.784
1001,25728.0,0.197859,0.272286,0.080,0.084000,0.08500,0.093000,1.782
1002,25728.0,0.208056,0.265281,0.000,0.062000,0.12700,0.273000,4.117
1003,25728.0,0.633739,0.479202,0.000,0.328000,0.51700,0.759250,6.003
1004,25728.0,0.984297,0.954003,0.000,0.340000,0.64900,1.275000,8.493
1005,25728.0,0.624357,0.638267,0.000,0.201000,0.40400,0.777000,6.184
1006,25728.0,0.447727,0.641061,0.000,0.067000,0.18050,0.515000,7.655
1007,25728.0,0.728270,0.745559,0.000,0.252000,0.42000,0.908000,8.040
1008,25728.0,0.732090,0.652383,0.047,0.360000,0.48500,0.774250,7.916


# Convert Daycodes to Dates

In [70]:
hoursonly = [x-int(x/100)*100 for x in dates_df]
a_df = pd.DataFrame()
a_df['hours'] = hoursonly

In [153]:
def get_date(num):
    year_list = [2009, 2010, 2011, 2012]
    day_list= [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    month_list = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    day = None
    day_sum = 0
    month = 1
    year = 2009
    for year in year_list:
        for month, days_of_month in zip(month_list, day_list):
            if (day_sum+days_of_month)>num:
                day = num-day_sum
                month__ = month
                year__ = year
                break
            elif (day_sum+days_of_month)==num:
                day = days_of_month
                month__ = month
                year__ = year
                break
            else:
                day_sum = day_sum+days_of_month
        if day !=None:
            break
    return(day, month__, year__)

In [154]:
get_date(730)

(31, 'Dec', 2010)

In [155]:
dt = datetime.datetime(2009, 7, 14, 0, 0, 0)
end = datetime.datetime(2010, 12, 31, 23, 59, 59)
step = datetime.timedelta(minutes = 30)

dates = []

while dt < end:
    dates.append(dt.strftime('%Y-%m-%d %H:%M:%S'))
    dt += step

In [159]:
tms.columns = dates

In [162]:
tms.to_csv(path+'Final_Timeseries.csv')